In [1]:
import os

import numpy as np
import pandas as pd
import shap
import xgboost as xgb
from sklearn.preprocessing import StandardScaler

import mgitools.os_helpers as os_helpers

In [2]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [3]:
result_dir = '../results/01282021_new'
from pathlib import Path
Path(result_dir).mkdir(parents=True, exist_ok=True)

In [4]:
combined_fp = '../data/formatted/aggregated_01282022.txt.gz'

In [5]:
combined = pd.read_csv(combined_fp, sep='\t', index_col=0)

Columns (37081) have mixed types.Specify dtype option on import or set low_memory=False.


In [6]:
d = pd.read_csv('../data/new/199_driver_genes.txt', sep='\t')
d

,Gene,Tumor suppressor or oncogene prediction (by 20/20+)
0,PHF6,possible tsg
1,ABL1,NaN
2,ALK,NaN
3,AR,NaN
4,ARAF,NaN
...,...,...
183,KMT2A,tsg
184,KMT2B,tsg
185,MAX,oncogene
186,MED12,oncogene


In [7]:
target_genes = sorted(set(d['Gene']))

###### protein pairs

In [8]:
pathways = pd.read_csv('../data/new/protein_pair_table_v2.txt', sep='\t')
pathways

,GENE,SUB_GENE,pair_pro,SUB_GENE.is_TF_downstream,SUB_GENE.is_TF_upstream,SUB_GENE.is_kinase_substrate,SUB_GENE.is_phosphatase_substrate,SUB_GENE.is_upstream_kinase,SUB_GENE.is_upstream_phosphatase,SUB_GENE.is_complex_partner
0,TP53,CDKN1A,TP53:CDKN1A,True,False,False,False,False,False,False
1,TP53,SIAH1,TP53:SIAH1,True,False,False,False,False,False,False
2,TP53,SFN,TP53:SFN,True,False,False,False,False,False,False
3,TP53,RPRM,TP53:RPRM,True,False,False,False,False,False,False
4,TP53,GADD45A,TP53:GADD45A,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
831929,SETD2,SETD2,SETD2:SETD2,False,False,False,False,False,False,False
831930,PUMA,PUMA,PUMA:PUMA,False,False,False,False,False,False,False
831931,NOXA,NOXA,NOXA:NOXA,False,False,False,False,False,False,False
831932,FOXR2,FOXR2,FOXR2:FOXR2,False,False,False,False,False,False,False


In [9]:
pathways[[True if g in target_genes else False
         for g in pathways['GENE']]]

,GENE,SUB_GENE,pair_pro,SUB_GENE.is_TF_downstream,SUB_GENE.is_TF_upstream,SUB_GENE.is_kinase_substrate,SUB_GENE.is_phosphatase_substrate,SUB_GENE.is_upstream_kinase,SUB_GENE.is_upstream_phosphatase,SUB_GENE.is_complex_partner
0,TP53,CDKN1A,TP53:CDKN1A,True,False,False,False,False,False,False
1,TP53,SIAH1,TP53:SIAH1,True,False,False,False,False,False,False
2,TP53,SFN,TP53:SFN,True,False,False,False,False,False,False
3,TP53,RPRM,TP53:RPRM,True,False,False,False,False,False,False
4,TP53,GADD45A,TP53:GADD45A,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
830817,IDH1,IDH1,IDH1:IDH1,False,False,False,False,False,False,False
831488,BCOR,BCOR,BCOR:BCOR,False,False,False,False,False,False,False
831552,ATRX,ATRX,ATRX:ATRX,False,False,False,False,False,False,False
831798,RQCD1,RQCD1,RQCD1:RQCD1,False,False,False,False,False,False,False


In [10]:
gene_to_subgenes = {t:[g for g in sorted(set(pathways[pathways['GENE']==t]['SUB_GENE'])) if g != t]
                    for t in target_genes}
target_genes[0], len(gene_to_subgenes[target_genes[0]])

('ABL1', 126)

In [11]:
combined

,AAAS_proteome,AAK1_proteome,AATF_proteome,ABCA1_proteome,ABCA2_proteome,ABCB1_proteome,ABCB11_proteome,ABCC2_proteome,ABCC3_proteome,ABCE1_proteome,...,TOMM34_S186|TOMM34|NP_006800.2_S186s_1_1_186_186_phospho,XYLB_S354|XYLB|NP_001336107.1_S354s_1_1_354_354_phospho,NARS1_S88|NARS1|NP_004530.1_S88s_1_1_88_88_phospho,GPR65_S324|GPR65|NP_003599.2_S324s_1_1_324_324_phospho,DBI_S2|DBI|NP_001073331.1_S2s_1_1_2_2_phospho,SNRPD3_S2|SNRPD3|NP_001265585.1_S2s_1_1_2_2_phospho,RAB28_S207|RAB28|NP_001017979.1_S207s_1_1_207_207_phospho,FBXL7_S105|FBXL7|NP_036436.1_S105s_1_1_105_105_phospho,MAP3K14_S410|MAP3K14|NP_003945.2_S410s_1_1_410_410_phospho,UGGT2_S952|UGGT2|NP_064506.3_S952s_1_1_952_952_phospho
01BR001-T,0.35140,-0.00390,1.10105,0.98196,NaN,NaN,NaN,NaN,-3.91029,1.12252,...,-0.21957,-2.064230,-1.22276,-0.40442,0.54564,0.875650,3.109490,-3.34221,-2.561220,-0.143310
01BR008-T,0.53624,-0.24927,1.54379,0.85463,NaN,NaN,NaN,NaN,-1.42230,0.51320,...,0.42100,-1.774430,-1.45275,2.80914,-2.57344,-0.250530,0.409140,-1.38308,-0.397280,1.701790
01BR009-T,-0.27902,-0.33482,0.69390,0.78852,NaN,NaN,NaN,NaN,-1.49698,-0.39790,...,-0.41210,-1.341260,-2.09601,-0.53712,1.80276,0.476930,1.350520,-1.68391,-1.756450,2.011120
01BR010-T,0.35560,-0.05965,-0.06194,-2.39742,NaN,NaN,NaN,4.85220,1.09662,-0.41295,...,-0.20548,-0.859580,-1.73799,-0.80625,-0.82664,0.986640,0.330970,1.13409,-0.707344,1.794460
01BR015-T,0.95351,-0.37789,-0.68548,-3.96565,NaN,NaN,NaN,NaN,-5.95178,0.12743,...,-0.10312,-1.371280,-1.56777,-0.28762,-0.17280,0.671700,0.437580,-1.53015,-1.262580,-2.419064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N-04280-T,-0.54060,0.28620,0.56570,-0.29791,NaN,NaN,NaN,NaN,NaN,0.45859,...,0.83508,-1.278484,-0.66452,-1.30025,0.42640,0.582466,1.899430,-0.80850,1.863990,-0.550450
C3N-04282-T,-0.38949,0.11637,0.56524,-1.92610,NaN,NaN,NaN,-1.70998,1.13998,-0.12112,...,-1.51760,0.610720,1.00393,-1.01017,-0.07182,-0.015610,-0.499620,-2.86815,0.521480,-0.113980
C3N-04283-T,0.38921,-0.15259,-0.18576,1.35119,NaN,NaN,0.25874,0.90227,0.31181,-0.05971,...,-1.09135,0.213746,-0.55999,-2.71406,-0.63692,0.084090,3.855510,-1.65134,0.883820,-0.814058
C3N-04284-T,0.76318,0.31274,0.45277,-1.21128,NaN,NaN,NaN,NaN,0.54146,1.09926,...,-0.26326,0.822460,1.72946,-1.00419,-1.34322,0.639012,-2.874184,-0.22867,-0.249884,-2.721020


In [12]:
{c.split('_')[-1] for c in combined.columns}

{'1',
 '2',
 '3',
 "3'UTR",
 '4',
 '5',
 "5'Flank",
 "5'UTR",
 '6',
 'AFR',
 'AMR',
 'DETERMINE',
 'Del',
 'EAS',
 'EUR',
 'FRAME',
 'INS',
 'Ins',
 'Intron',
 'Mutation',
 'RNA',
 'SAS',
 'SNP',
 'Silent',
 'Site',
 'TumorPurity',
 'age',
 'cnv',
 'disease',
 'expression',
 'female',
 'germline',
 'mutation',
 'phospho',
 'proteome'}

#### train ols

In [13]:
combined.columns

Index(['AAAS_proteome', 'AAK1_proteome', 'AATF_proteome', 'ABCA1_proteome',
       'ABCA2_proteome', 'ABCB1_proteome', 'ABCB11_proteome', 'ABCC2_proteome',
       'ABCC3_proteome', 'ABCE1_proteome',
       ...
       'TOMM34_S186|TOMM34|NP_006800.2_S186s_1_1_186_186_phospho',
       'XYLB_S354|XYLB|NP_001336107.1_S354s_1_1_354_354_phospho',
       'NARS1_S88|NARS1|NP_004530.1_S88s_1_1_88_88_phospho',
       'GPR65_S324|GPR65|NP_003599.2_S324s_1_1_324_324_phospho',
       'DBI_S2|DBI|NP_001073331.1_S2s_1_1_2_2_phospho',
       'SNRPD3_S2|SNRPD3|NP_001265585.1_S2s_1_1_2_2_phospho',
       'RAB28_S207|RAB28|NP_001017979.1_S207s_1_1_207_207_phospho',
       'FBXL7_S105|FBXL7|NP_036436.1_S105s_1_1_105_105_phospho',
       'MAP3K14_S410|MAP3K14|NP_003945.2_S410s_1_1_410_410_phospho',
       'UGGT2_S952|UGGT2|NP_064506.3_S952s_1_1_952_952_phospho'],
      dtype='object', length=42079)

In [20]:
def get_data_for_pair(gene, subgene, combined, features=['cnv', 'is_mutated', 'is_pathogenic_germline'],
                     standalone=['TumorPurity'], target='proteome', scale_features=['TumorPurity', 'cnv'],
                     scale_y=False):
    cols = [f'{gene}_{feat}' for feat in features]
    cols += standalone
    cols = sorted(set(cols))
    
    # check to make sure all columns are in X, if not then return None, None
    if f'{subgene}_{target}' not in combined.columns:
        return None, None
    if len([x for x in cols if x not in combined.columns]):
        return None, None
    
    # make training dataframe
    X = combined[cols].copy()
    y = combined[[f'{subgene}_{target}']].copy()
    
    # filter NA
    X = X.dropna()
    
    # exit if empty
    if not X.shape[0]:
        return None, None
    
    to_scale = [c for c in cols for f in scale_features if f in c]
    prev = X[[c for c in X.columns if c not in to_scale]]
    
    # scale features in scale_features
    if to_scale:
        scale = X[to_scale]
        scale = pd.DataFrame(data=StandardScaler().fit_transform(scale.values), columns=scale.columns, index=scale.index)
        X = pd.merge(scale, prev, left_index=True, right_index=True)
    
    # scale y if scale_y
    if scale_y:
        y = pd.DataFrame(data=StandardScaler().fit_transform(y.values), columns=y.columns, index=y.index)
    
    # if we aren't all numerical exit
    try:
        X = X.astype(np.float32)
    except ValueError:
        return None, None
    
    y = y.loc[X.index]
    # make sure no y nulls
    y = y.dropna()
    X = X.loc[y.index]
    
    # replace with gene name so more readable
    X.columns = [c.replace(gene, 'driver_gene') for c in X.columns]

    return X, y
    

In [21]:
import statsmodels.api as sm
from patsy import dmatrices
def run_ols(X, y):
    ground = y.values.flatten()
    df = pd.merge(X, y, left_index=True, right_index=True)

    command = f'{y.columns[0]} ~ ' + ' + '.join(X.columns)
    y_sm, X_sm = dmatrices(command, data=df, return_type='dataframe')
    
    model = sm.OLS(y_sm, X_sm)
    results = model.fit()
    
    coef_df = pd.DataFrame.from_dict({'coef': results.params.to_list(),
                           'p-value': results.pvalues.to_list()})
    coef_df.index = results.params.index.to_list()
    
    return {'result': results, 'coef_df': coef_df, 'r-squared': results.rsquared_adj,
           'r-squared p-value': results.f_pvalue, 'groundtruth': ground, 'predicted': results.fittedvalues.to_list(),
           'features': results.params.index.to_list(), 'X': X, 'y': y}

select model parameters here

['cnv', 'has_nonsilent_mutation', 'is_pathogenic_germline', 'TumorPurity', 'expression', 'proteome', 'phospho']


In [22]:
# features = []
# standalone = ['TumorPurity',
#  'clinical_is_tumor',
#  'clinical_age',
#  'clinical_is_female',
#  'clinical_predicted_ancestry_is_AFR',
#  'clinical_predicted_ancestry_is_AMR',
#  'clinical_predicted_ancestry_is_EAS',
#  'clinical_predicted_ancestry_is_EUR',
#  'clinical_predicted_ancestry_is_SAS']
# target = 'proteome'
# scale_features = []
# scale_y = False

In [51]:
features = ['cnv', 'has_nonsilent_mutation']
standalone = ['TumorPurity']
target = 'phospho'
scale_features = ['TumorPurity', 'cnv']
scale_y = False

In [52]:
# features = ['cnv', 'has_nonsilent_mutation', 'is_pathogenic_germline']
# standalone = ['TumorPurity']
# target = 'expression'
# scale_features = ['TumorPurity', 'cnv']
# scale_y = False

In [53]:
# features = ['cnv', 'has_nonsilent_mutation', 'is_pathogenic_germline']
# standalone = ['TumorPurity',
#              'methylation_subtype_1',
#              'methylation_subtype_2',
#              'methylation_subtype_3',
#              'methylation_subtype_4',
#              'methylation_subtype_5',
#              'methylation_subtype_6']
# target = 'proteome'
# scale_features = ['TumorPurity', 'cnv']
# scale_y = False

run this cell for proteome and expression target

In [40]:
results_dict = {}
for disease in sorted({x for x in combined['disease'] if not pd.isnull(x)}):
    print(disease)
    results_dict[disease] = {}
    filtered = combined[combined['disease']==disease]
    for i, (gene, subgenes) in enumerate(gene_to_subgenes.items()):
        if i % 10 == 0:
            print(i, gene)
        genes = [gene]
        genes += subgenes
        genes = sorted(set(genes))
        for subgene in genes:
            X, y = get_data_for_pair(gene, subgene, filtered, features=features,
                                    standalone=standalone, target=target, scale_features=scale_features,
                                    scale_y=scale_y)
            # make sure features have no -
            if X is not None and X.shape[0] and '-' not in gene and '-' not in subgene:

                X.columns = [c.replace('-', '_') for c in X.columns]
                r = run_ols(X.copy(), y.copy())
                results_dict[disease][f'{gene}_{subgene}'] = r
    
            

BR
0 ABL1
10 ARID1A
20 BCL2


invalid value encountered in double_scalars
invalid value encountered in double_scalars


30 CDK4
40 CSDE1


invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars


50 EGR3


...

invalid value encountered in double_scalars
invalid value encountered in double_scalars


ccRCC
0 ABL1
10 ARID1A
20 BCL2
120 NFE2L2
130 PHF6


invalid value encountered in double_scalars
invalid value encountered in double_scalars


140 PPP2R1A
150 RET
160 SETD2
170 STK11
180 TP53


invalid value encountered in double_scalars
invalid value encountered in double_scalars


run this one for phospho


In [54]:
[x for x in combined.columns if 'phospho' in x][:10]

['AHNAK_S93|AHNAK|NP_001333374.1_S93s_1_1_93_93_phospho',
 'AHNAK_S135|AHNAK|NP_001333374.1_S135s_1_1_135_135_phospho',
 'AHNAK_S216|AHNAK|NP_001333374.1_S216s_1_1_216_216_phospho',
 'AHNAK_S379|AHNAK|NP_001333374.1_S379s_1_1_379_379_phospho',
 'AHNAK_T490|AHNAK|NP_001333374.1_T490t_1_1_490_490_phospho',
 'AHNAK_S511|AHNAK|NP_001333374.1_S511s_1_1_511_511_phospho',
 'AHNAK_S819|AHNAK|NP_001333374.1_S819s_1_1_819_819_phospho',
 'AHNAK_S886|AHNAK|NP_001333374.1_S886s_1_1_886_886_phospho',
 'AHNAK_S1010|AHNAK|NP_001333374.1_S1010s_1_1_1010_1010_phospho',
 'AHNAK_S1042|AHNAK|NP_001333374.1_S1042s_1_1_1042_1042_phospho']

In [55]:
results_dict = {}
x, y = None, None
for disease in sorted({x for x in combined['disease'] if not pd.isnull(x)}):
    print(disease)
    results_dict[disease] = {}
    filtered = combined[combined['disease']==disease]
    for i, (gene, subgenes) in enumerate(gene_to_subgenes.items()):
        if i % 10 == 0:
            print(i, gene)
        genes = [gene]
        genes += subgenes
        genes = sorted(set(genes))
        for subgene in genes:
            sites = ['_'.join(s.split('_')[:-1]) for s in filtered.columns if subgene in s or gene in s]
            for site in sites:
                X, y = get_data_for_pair(gene, site, filtered, features=features,
                                        standalone=standalone, target=target, scale_features=scale_features,
                                        scale_y=scale_y)
                x, y = (X, y)
                if X is not None and y is not None:
                    y.columns = [y.columns[0].replace('-', '_').replace(' ', '_').replace('|', '_').replace('.', '_').replace(';', '_')]
                if X is not None and X.shape[0] and '-' not in gene and '-' not in subgene:
                    X.columns = [c.replace('-', '_') for c in X.columns]
#                     print(X.columns, y.columns)
                    r = run_ols(X.copy(), y.copy())
                    results_dict[disease][f'{gene}_{site}'] = r

BR
0 ABL1
10 ARID1A
20 BCL2
30 CDK4
40 CSDE1
50 EGR3
60 FAT1
70 GNAQ
80 IDH1
90 KIF1A
100 MACF1
110 MGMT
120 NFE2L2
130 PHF6
140 PPP2R1A
150 RET
160 SETD2
170 STK11
180 TP53
CO
0 ABL1
10 ARID1A
20 BCL2
30 CDK4
40 CSDE1
50 EGR3
60 FAT1
70 GNAQ
80 IDH1
90 KIF1A
100 MACF1
110 MGMT
120 NFE2L2
130 PHF6
140 PPP2R1A
150 RET
160 SETD2
170 STK11
180 TP53
GBM
0 ABL1
10 ARID1A
20 BCL2
30 CDK4
40 CSDE1
50 EGR3
60 FAT1
70 GNAQ
100 MACF1
110 MGMT
120 NFE2L2
130 PHF6
140 PPP2R1A
150 RET
160 SETD2
170 STK11
180 TP53
HNSCC
0 ABL1
10 ARID1A
20 BCL2
30 CDK4
40 CSDE1
50 EGR3
60 FAT1
70 GNAQ
80 IDH1
90 KIF1A
100 MACF1
110 MGMT
120 NFE2L2
130 PHF6
140 PPP2R1A
150 RET
160 SETD2
170 STK11
180 TP53
LSCC
0 ABL1
10 ARID1A
20 BCL2
30 CDK4
40 CSDE1
50 EGR3
60 FAT1
70 GNAQ
80 IDH1
90 KIF1A
100 MACF1
110 MGMT
120 NFE2L2
130 PHF6
140 PPP2R1A
150 RET
160 SETD2
170 STK11
180 TP53
LUAD
0 ABL1
10 ARID1A
20 BCL2
30 CDK4
40 CSDE1
50 EGR3
60 FAT1
70 GNAQ
80 IDH1
90 KIF1A
100 MACF1
110 MGMT
120 NFE2L2
130 PHF6
140 PPP2R1A
15

adjust p-values

In [56]:
# adjust p-values for fdr in both R and coefs
import statsmodels
for disease, results in results_dict.items():
    print(disease)
    if results:
        keys, ps = zip(*[(k, d['r-squared p-value']) for k, d in results.items()
                         if not pd.isnull(d['r-squared p-value'])])
        corrected = statsmodels.stats.multitest.fdrcorrection(ps)[1]
        for k, c in zip(keys, corrected):
            results[k]['r-squared FDR'] = c
        for k, d in results.items():
            if 'r-squared FDR' not in d:
                d['r-squared FDR'] = np.nan

        xs, ls, feats = [], [], []
        for feat in list(results.values())[0]['features']:
            temp_xs, temp_ls = zip(*[(k, d['coef_df'].loc[feat, 'p-value']) for k, d in results.items()
                          if feat in d['features']])
            xs += temp_xs
            ls += temp_ls
            feats += [feat] * len(temp_xs)

        keys, ps, fts = zip(*[(x, l, f) for x, l, f in zip(xs, ls, feats) if not pd.isnull(l)])
        corrected = statsmodels.stats.multitest.fdrcorrection(ps)[1]
        for k, c, feat in zip(keys, corrected, fts):
            if 'FDR' not in results[k]['coef_df'].columns:
                results[k]['coef_df']['FDR'] = np.nan
                results[k]['coef_df']['-log10(FDR)'] = np.nan
            else:
                results[k]['coef_df'].loc[feat, 'FDR'] = c
                results[k]['coef_df'].loc[feat, '-log10(FDR)'] = -np.log10(c)

    

BR
CO
GBM
HNSCC
LSCC
LUAD
OV
PDA
UCEC
ccRCC


In [57]:
# results_dict['BR']['TP53_TP53']['result'].summary()

In [58]:
# results_dict['BR']['TP53_TP53']['coef_df']

In [59]:
import json
def save_results_dict(filepath, results_dict):
    save_dict = {}
    for disease, d in results_dict.items():
        save_dict[disease] = {}
        for pair, d2 in d.items():
            save_dict[disease][pair] = {}
            for k, val in d2.items():
                if isinstance(val, pd.DataFrame):
                    save_dict[disease][pair][k] = val.to_dict()
                elif 'numpy.ndarray' in str(type(val)) and val.size<=1 and np.isnan(val):
                    save_dict[disease][pair][k] = None
                elif 'numpy.ndarray' in str(type(val)):
#                     print(val, str(type(val)), val.size)
                    if len(val):
                        save_dict[disease][pair][k] = [float(v) for v in val]
                elif 'numpy.float' in str(type(val)):
                    save_dict[disease][pair][k] = float(val)
                elif isinstance(val, statsmodels.regression.linear_model.RegressionResultsWrapper):
                    pass
                else:
                    save_dict[disease][pair][k] = val
    json.dump(save_dict, open(filepath, 'w'))
    
def load_results_dict(filepath):
    loaded = json.load(open(filepath))
    results_dict = {}
    for disease, d in loaded.items():
        results_dict[disease] = {}
        for pair, d2 in d.items():
            results_dict[disease][pair] = {}
            for k, val in d2.items():
                if k in ['coef_df', 'X', 'y']:
                    results_dict[disease][pair][k] = pd.DataFrame.from_dict(val)
                elif k in ['groundtruth', 'predicted', 'features']:
                    results_dict[disease][pair][k] = np.asarray(val)
                else:
                    results_dict[disease][pair][k] = val
    return results_dict

In [60]:
run_dir = os.path.join(result_dir, f'som_germ_cnv_pur_target_{target}')
Path(run_dir).mkdir(exist_ok=True, parents=True)
run_dir

'../results/01282021_new/som_germ_cnv_pur_target_phospho'

In [61]:
filepath = os.path.join(run_dir, 'results.json')
save_results_dict(filepath, results_dict)

In [62]:
# also save metadata
summary = {
    'features': features,
    'standalone': standalone,
    'target': target,
    'scale_features': scale_features,
    'scale_target': scale_y,
    'input_source': combined_fp,
}
json.dump(summary, open(os.path.join(run_dir, 'summary.json'), 'w'))

In [63]:
def get_master_coef_df(results_dict):
    master_coef_dict = None
    for disease, results in results_dict.items():
        for k, r in results.items():
            pieces = k.split('_')[0], '_'.join(k.split('_')[1:])
            driver, subgene = pieces
            n = r['coef_df'].shape[0]-1
            df_dict = r['coef_df'].iloc[1:, :].to_dict()
            order = r['coef_df'].index[1:].to_list()
            for k, v in df_dict.items():
                df_dict[k] = [v[o] for o in order]
            df_dict['feature'] = order

            # replace 1. fdr with np.nan
            if 'FDR' in df_dict:
                df_dict['FDR'] = [np.nan if pd.isnull(p) else x
                                  for p, x in zip(df_dict['p-value'], df_dict['FDR'])]
                df_dict['-log10(FDR)'] = [np.nan if pd.isnull(p) else x
                                          for p, x in zip(df_dict['p-value'], df_dict['-log10(FDR)'])]
            else:
                df_dict['FDR'] = [np.nan for i in range(n)]
                df_dict['-log10(FDR)'] = [np.nan for i in range(n)]
                
            df_dict['driver'] = [driver for i in range(n)]
            df_dict['target'] = [subgene for i in range(n)]
            df_dict['disease'] = [disease for i in range(n)]
            df_dict['model_r2'] = [r['r-squared'] for i in range(n)]
            df_dict['model_r2_FDR'] = [r['r-squared FDR'] for i in range(n)]

            if master_coef_dict is None:
                master_coef_dict = df_dict
            else:
                for k in master_coef_dict.keys():
                    master_coef_dict[k] += df_dict[k]
    master_coef_df = pd.DataFrame.from_dict(master_coef_dict)     
    master_coef_df.index = np.arange(master_coef_df.shape[0])
    
    return master_coef_df

In [64]:
master_coef_df = get_master_coef_df(results_dict)
master_coef_df.to_csv(os.path.join(run_dir, 'coef_results.txt'), sep='\t', index=False)

In [65]:
master_coef_df

,coef,p-value,FDR,-log10(FDR),feature,driver,target,disease,model_r2,model_r2_FDR
0,0.164722,2.607030e-06,8.035901e-06,5.094965,driver_gene_cnv,ABL1,ABL1_S588|ABL1|NP_009297.2_S588s_1_1_588_588,BR,0.090911,7.853147e-08
1,0.135048,1.340333e-04,3.421983e-04,3.465722,TumorPurity,ABL1,ABL1_S588|ABL1|NP_009297.2_S588s_1_1_588_588,BR,0.090911,7.853147e-08
2,0.698709,9.974332e-03,1.878631e-02,1.726159,driver_gene_has_nonsilent_mutation,ABL1,ABL1_S588|ABL1|NP_009297.2_S588s_1_1_588_588,BR,0.090911,7.853147e-08
3,0.176910,1.273530e-05,3.663727e-05,4.436077,driver_gene_cnv,ABL1,ABL1_T871|ABL1|NP_009297.2_T871t_1_1_871_871,BR,0.053451,6.982833e-05
4,-0.013966,7.307202e-01,7.850275e-01,0.105115,TumorPurity,ABL1,ABL1_T871|ABL1|NP_009297.2_T871t_1_1_871_871,BR,0.053451,6.982833e-05
...,...,...,...,...,...,...,...,...,...,...
503545,0.046613,1.907153e-01,2.474452e-01,0.606521,driver_gene_cnv,ZNF750,ZNF740_S44|ZNF740|NP_001004304.1_S44s_1_1_44_44,ccRCC,0.043374,3.883592e-04
503546,0.722028,5.261574e-02,7.809862e-02,1.107357,driver_gene_has_nonsilent_mutation,ZNF750,ZNF740_S44|ZNF740|NP_001004304.1_S44s_1_1_44_44,ccRCC,0.043374,3.883592e-04
503547,0.318029,6.391262e-19,3.510270e-18,17.454659,TumorPurity,ZNF750,ZNF777_S604|ZNF777|NP_056509.2_S604s_1_1_604_604,ccRCC,0.207376,5.060838e-18
503548,-0.011898,7.605060e-01,8.045909e-01,0.094425,driver_gene_cnv,ZNF750,ZNF777_S604|ZNF777|NP_056509.2_S604s_1_1_604_604,ccRCC,0.207376,5.060838e-18
